# Interactive Network Visualization of Game of Thrones Characters

This notebook demonstrates how to create an interactive network visualization of character relationships from Game of Thrones using the book1.csv dataset.

## 1. Setup and Loading Required Libraries

First, let's import all the necessary libraries:

In [1]:
# Import libraries for data handling
import pandas as pd
import numpy as np

# Import network libraries
import networkx as nx

# Import visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode

# For color mapping
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Display plots inline
init_notebook_mode(connected=True)

## 2. Loading and Processing the Network Data

Let's load the Game of Thrones character network data from book1.csv. This dataset contains character interactions from the first book.

In [2]:
# Load the edge list from CSV
file_path = "data/got/book1.csv"
edges_df = pd.read_csv(file_path)

# Display the first few rows to understand the data structure
print(f"Dataset shape: {edges_df.shape}")
edges_df.head()

Dataset shape: (684, 5)


,Source,Target,Type,weight,book
0,Addam-Marbrand,Jaime-Lannister,Undirected,3,1
1,Addam-Marbrand,Tywin-Lannister,Undirected,6,1
2,Aegon-I-Targaryen,Daenerys-Targaryen,Undirected,5,1
3,Aegon-I-Targaryen,Eddard-Stark,Undirected,4,1
4,Aemon-Targaryen-(Maester-Aemon),Alliser-Thorne,Undirected,4,1


## 3. Creating a Network Representation with NetworkX

Now we'll create a network representation using NetworkX. We'll treat characters as nodes and their interactions as edges.

In [3]:
# Create a graph
G = nx.Graph()

# Add edges and their weights to the graph
for _, row in edges_df.iterrows():
    source = row['Source']
    target = row['Target']
    weight = row['weight'] if 'weight' in row else 1
    
    # Add edge with weight
    G.add_edge(source, target, weight=weight)

# Print basic network statistics
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 187
Number of edges: 684


### 3.1 Calculate Network Metrics

Let's calculate some network metrics that will help us visualize the network more effectively:
- Degree centrality: to size the nodes
- Community detection: to color the nodes

In [4]:
# Calculate degree for each node
degree = dict(G.degree())

# Detect communities using Louvain algorithm
try:
    import community as community_louvain
    communities = community_louvain.best_partition(G)
except ImportError:
    # Fallback to NetworkX's community detection
    from networkx.algorithms import community
    communities_sets = community.greedy_modularity_communities(G)
    communities = {}
    for i, comm in enumerate(communities_sets):
        for node in comm:
            communities[node] = i

# Calculate betweenness centrality for node sizing
betweenness = nx.betweenness_centrality(G)

# Create a node attribute dictionary
node_attributes = {
    node: {
        'degree': degree[node],
        'community': communities[node] if node in communities else 0,
        'betweenness': betweenness[node]
    } for node in G.nodes()
}

# Add attributes to the graph
nx.set_node_attributes(G, node_attributes)

# Define a single colormap to use throughout the notebook
num_communities = len(set(communities.values()))
# Using Plotly's qualitative color sequences for consistent coloring
network_colormap = px.colors.qualitative.D3[:num_communities]

In [5]:
pos = nx.spring_layout(G, k=1.00, iterations=500, seed=42)

## 4. Zoom-Dependent Label Visibility

To achieve truly dynamic label visibility that responds to zoom level, we need to use Plotly's FigureWidget with custom callback functions. This advanced feature allows labels to appear only when zoomed in to specific areas.

In [6]:
import plotly.graph_objects as go
from plotly.graph_objs import FigureWidget

def create_zoom_aware_network_widget(G, pos, zoom_threshold=0.8):
    node_x = []
    node_y = []
    node_color = []
    node_size = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        # Use the community ID to get color from the global colormap
        community_id = G.nodes[node]['community']
        node_color.append(network_colormap[community_id % len(network_colormap)])
        node_size.append(10 + G.nodes[node]['betweenness'] * 100)
        node_text.append(node)

    # Get edge weights to calculate min and max for scaling
    edge_weights = [data.get('weight', 1) for _, _, data in G.edges(data=True)]
    min_weight = min(edge_weights) if edge_weights else 1
    max_weight = max(edge_weights) if edge_weights else 1
    
    # Define min and max edge width for better aesthetics
    min_edge_width = 0.5
    max_edge_width = 4.0
    
    # Prepare edge data (edges first, so nodes are on top)
    edge_traces = []
    for source, target, data in G.edges(data=True):
        x0, y0 = pos[source]
        x1, y1 = pos[target]
        weight = data.get('weight', 1)
        
        # Scale edge width between min and max width based on weight
        if max_weight > min_weight:  # Avoid division by zero
            normalized_weight = (weight - min_weight) / (max_weight - min_weight)
            width = min_edge_width + normalized_weight * (max_edge_width - min_edge_width)
        else:
            width = min_edge_width
            
        edge_traces.append(go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            line=dict(width=width, color='rgba(150,150,150,0.75)'),
            hoverinfo='none',
            mode='lines'
        ))

    # Node trace with labels initially hidden
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        marker=dict(
            size=node_size,
            color=node_color,
            line=dict(width=1, color='rgba(50,50,50,0.8)'),
            opacity=1  # Setting node opacity to fully opaque
        ),
        text=node_text,
        textposition='top center',
        textfont=dict(size=10, color='black'),
        hoverinfo='text',
        showlegend=False,
        visible=True
    )
    node_trace.textfont.color = 'rgba(0,0,0,0)'

    fig = FigureWidget(data=edge_traces + [node_trace])
    fig.update_layout(
        title='Game of Thrones Character Network with Zoom-Dependent Labels',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=1200,
        height=800,
        paper_bgcolor='white',
        plot_bgcolor='white'
    )


    # Parameters for font size adjustment - now tied to zoom_threshold
    min_font_size = 8
    max_font_size = 14
    min_zoom_span = zoom_threshold * 0.25  # Very zoomed in (25% of threshold)
    max_zoom_span = zoom_threshold * 2.5   # Not zoomed in much (250% of threshold)
    
    def update_labels(trace, points, state):
        x_range = fig.layout.xaxis.range
        y_range = fig.layout.yaxis.range
        if x_range and y_range:
            x_span = abs(x_range[1] - x_range[0])
            y_span = abs(y_range[1] - y_range[0])
            
            # Use the average of x and y span as our zoom measure
            zoom_span = (x_span + y_span) / 2
            
            if zoom_span < zoom_threshold:
                # Calculate dynamic font size based on zoom level
                if zoom_span <= min_zoom_span:
                    # Maximum font size when very zoomed in
                    font_size = max_font_size
                elif zoom_span >= max_zoom_span:
                    # Minimum font size when zoomed out
                    font_size = min_font_size
                else:
                    # Linear scaling between min and max font size
                    zoom_ratio = (zoom_span - min_zoom_span) / (max_zoom_span - min_zoom_span)
                    font_size = max_font_size - zoom_ratio * (max_font_size - min_font_size)
                
                fig.data[-1].textfont.color = 'black'
                fig.data[-1].textfont.size = font_size
            else:
                # Hide labels when zoomed out too far
                fig.data[-1].textfont.color = 'rgba(0,0,0,0)'

    fig.layout.on_change(update_labels, 'xaxis', 'yaxis')

    return fig

# Example usage:
zoom_aware_fig_widget = create_zoom_aware_network_widget(G, pos, zoom_threshold=0.8)
zoom_aware_fig_widget

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgba(150,150,150,0.75)', 'width': 0.5},
              'mode': 'lines',
              'type': 'scatter',
              'uid': 'a25cbc7c-d414-487a-a8fa-a79ea03cbabd',
              'x': [0.22497279778933718, 0.010251626732481109, None],
              'y': [-0.08496436427478211, -0.04380961742058167, None]},
             {'hoverinfo': 'none',
              'line': {'color': 'rgba(150,150,150,0.75)', 'width': 0.5364583333333334},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '428540b6-1157-4692-bdd2-8a3fcf81c230',
              'x': [0.22497279778933718, 0.037115510346755934, None],
              'y': [-0.08496436427478211, -0.04556504226486564, None]},
             {'hoverinfo': 'none',
              'line': {'color': 'rgba(150,150,150,0.75)', 'width': 0.5243055555555556},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '72582cb2-51